# LlamaIndex Bottoms-Up Development - LLMs and Prompts
This notebook walks through testing an LLM using the primary prompt templates used in llama-index.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Setup
In this section, we load a test document, create an LLM, and copy prompts from llama-index to test with.

First, let's load a quick document to test with. Right now, we will just load it as plain text, but we can do other operations later!

In [2]:
with open("./data/getting_started/starter_example.md", "r") as f:
    text = f.read()

Next, we create our LLM!

In [5]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

LlamaIndex uses some simple templates under the hood for answering queries -- mainly a `text_qa_template` for obtaining initial answers, and a `refine_template` for refining an existing answer when all the text does not fit into one LLM call.

Let's copy the default templates, and test out our LLM with a few questions.

In [7]:
from llama_index.core import PromptTemplate

text_qa_template = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_template = PromptTemplate(
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

Now, lets test a few questions!

## Text QA Template Testing

In [8]:
question = "How can I install llama-index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To install LlamaIndex, you can follow the steps outlined in the context information under the "Download" section. You can clone the LlamaIndex repository by running the following command in your terminal:

```bash
$ git clone https://github.com/jerryjliu/llama_index.git
```

After cloning the repository, you can navigate to the `llama_index` directory and verify its contents. This will allow you to access the examples and get started with using LlamaIndex.


In [9]:
question = "How do I create an index?"
prompt = text_qa_template.format(context_str=text, query_str=question)
response = llm.complete(prompt)
print(response.text)

To create an index, you can follow these steps:

1. Download the LlamaIndex repository and navigate to the `examples/paul_graham_essay` folder.
2. Create a new `.py` file and import `VectorStoreIndex` and `SimpleDirectoryReader` from `llama_index`.
3. Load the documents from the `data` folder using `SimpleDirectoryReader` and build the index using `VectorStoreIndex.from_documents(documents)`.
4. Use the index to create a query engine with `index.as_query_engine()`.
5. Query the index by calling `query_engine.query("Your query here")`.

By following these steps, you can create and query an index using LlamaIndex.


In [10]:
question = "How do I create an index? Write your answer using only code."
prompt = text_qa_template.format(context_str=text, query_str=question)
response_gen = llm.stream_complete(prompt)
for response in response_gen:
    print(response.delta, end="")

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
```

## Refine Template Testing

In [11]:
question = "How do I create an index? Write your answer using only code."
existing_answer = """To create an index using LlamaIndex, you need to follow these steps:

1. Download the LlamaIndex repository by cloning it from GitHub.
2. Navigate to the `examples/paul_graham_essay` folder in the cloned repository.
3. Create a new Python file and import the necessary modules: `VectorStoreIndex` and `SimpleDirectoryReader`.
4. Load the documents from the `data` folder using `SimpleDirectoryReader('data').load_data()`.
5. Build the index using `VectorStoreIndex.from_documents(documents)`.
6. To persist the index to disk, use `index.storage_context.persist()`.
7. To reload the index from disk, use the `StorageContext` and `load_index_from_storage` functions.

Note: This answer assumes that you have already installed LlamaIndex and have the necessary dependencies."""
prompt = refine_template.format(context_msg=text, query_str=question, existing_answer=existing_answer)
response = llm.complete(prompt)
print(response.text)

```python
from llama_index import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)
index.storage_context.persist()
```


### Chat Example
The LLM also has a `chat` method that takes in a list of messages, to simulate a chat session. 

In [16]:
from llama_index.core.llms import ChatMessage

chat_history = [
    ChatMessage(role="system", content="You are a helpful QA chatbot that can answer questions about llama-index."),
    ChatMessage(role="user", content="How do I create an index?"),
]

response = llm.chat(chat_history)
print(response.message)

assistant: To create an index, you can follow these general steps:

1. Define the purpose of your index: Determine what you want your index to represent and what data points you want to include.

2. Gather the data: Collect the necessary data points or information that you want to include in your index.

3. Calculate the index values: Depending on the type of index you are creating, you may need to calculate weighted averages, percentages, or other formulas to determine the values for each data point.

4. Normalize the data: If your data points are on different scales, you may need to normalize them to ensure they are comparable.

5. Combine the data points: Once you have calculated the values for each data point, combine them to create the overall index value.

6. Test and validate the index: Make sure that your index accurately reflects the data and meets the intended purpose.

7. Publish and maintain the index: Once your index is created, you can publish it for others to use and reg

## Conclusion

In this notebook, we covered the low-level LLM API, and tested out some basic prompts with out documentation data.